#  <span style="color: DeepPink"> AMES HOUSING</span>

# <span style="color: blue">EXPLORATORY DATA ANALYSIS ON AMES HOUSING</span>

### <span style="color: blue">Problem Statement: To be able to predict houses expected price.
Using the predictors,we can predict the selling price of the houses.AMES housing  data set contains 80 features excluding the target feature SalePrice.Using Exploratory Data Analysis and Feature Engineering and 
Model benchmarking was done with reference to Ridge, Lasso modelling methods.Since there are so many features we will choose the different features to build a model that can be used. Success of the model is based upon a few features that gives the lowest error.Primary audience is the home owners who wishes to sell their homes and home buyers who will like to purchase homes.</span>

  ## <span style="color: blue">IMPORTING LIBRARY</span>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
 

from sklearn.preprocessing import  StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer

from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet


from sklearn.metrics import mean_squared_error


%matplotlib inline

 ## <span style="color: blue">IMPORTING TRAIN DATASET</span>

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
test = pd.read_csv('test.csv')
test.head(2)


### <span style="color: blue">DATA CLEANING AND CHECKING FOR NULL VALUES IN TRAIN DATASET</span>


### <span style="color: blue"> There are many null values for both train and test datasets. </span>

In [ ]:
df.shape

In [ ]:
test.shape

In [ ]:
df.dtypes


### <span style="color: blue"> Some of these have a high percentage of nulls (such as ‘Pool Qc’ with 99.5% of values being nulls) and others have a low percentage. </span> 
 
 

In [ ]:
def missing(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    return missing_data.head(20)

missing(df)


### <span style="color: blue">SPLIT TRAIN DATASET TO  CATEGORICAL AND NUMERIC DATA</span>

In [ ]:
df.columns

In [ ]:
#### Separate into nominal features and temp features
#### 'Year Built','Year Remod/Add','Garage Yr Blt','Yr Sold' are the year sold will be analysing in the later part 

In [ ]:
nominal_features = ['Alley','Lot Shape','Land Contour',
                   'Overall Cond','Roof Style','Roof Matl','Exterior 1st',
                   'Exterior 2nd', 'Mas Vnr Type','Exter Qual',
                   'Exter Cond','Utilities','Lot Config','Land Slope',
                   'Neighborhood','Condition 1','Condition 2',
                   'Bldg Type','House Style','Overall Qual','Foundation','Bsmt Qual','Bsmt Cond',
                   'Bsmt Exposure','MS SubClass','MS Zoning','Street','BsmtFin Type 1','BsmtFin Type 2',
                   'Heating','Heating QC','Central Air','Electrical',
                   'Bsmt Full Bath','Bsmt Half Bath','Full Bath',
                   'Half Bath','Bedroom AbvGr','Kitchen AbvGr','Kitchen Qual',
                   'TotRms AbvGrd','Functional','Fireplaces',
                   'Fireplace Qu','Garage Type',
                   'Garage Finish','Garage Cars','Garage Qual',
                   'Garage Cond','Paved Drive','Pool QC','Fence',
                   'Misc Feature','Mo Sold',
                   'Sale Type','Year Built','Year Remod/Add','Garage Yr Blt','Yr Sold']

temp_features = ['Year Built','Year Remod/Add','Garage Yr Blt','Yr Sold']

non_numeric_features  = nominal_features + temp_features

# convert nominal features into category datatype
for features in nominal_features:
    df[features] = df[features].astype("category")

### <span style="color: blue"> CHECKING FOR NUMERICAL CATEGORY AND DROPPING CERTAIN COLUMNS</span>


#### <span style="color: blue"> DROP 2 COLUMNS THAT ARE PID AND ID AS THEY DOES NOT SHOW THE RELATIONSHIP WITH SALEPRICE</span>

In [ ]:
numerical_columns = [col for col in df.columns if col not in non_numeric_features]
numerical_columns = [col for col in numerical_columns if col not in ['PID','Id']]

In [ ]:
numerical_columns

In [ ]:
df[numerical_columns].dtypes

## <span style="color: blue"> Neighbourhoods relationship with Saleprice. As seen from the bar plot , StoneBr, NridgeHt, NoRidge and Grnhill are neighbourhoods with higher SalePrice.BrDale,IDOTRR, Meadowv are the neighbourhoods with least median prices.
</span>

In [ ]:
plt.figure(figsize=(15,10))
ax = df.groupby('Neighborhood').SalePrice.mean().sort_values(ascending=True).plot(kind='barh')
ax.set_xlabel('Sale Price')
ax.set_ylabel('Neighborhood')
ax.set_title('Neighborhood Mean Price')

## <span style="color: blue"> SHOWING THE BOXPLOT OF ALL THE NOMINAL FEATURES IN RELATIONSHIP WITH SALEPRICE. THERE ARE SOME OUTLIERS THAT CAN ALSO BE SEEN IN THE BOXPLOT
</span>

In [ ]:
def subplot_boxplot(dataframe, list_of_columns,reference='SalePrice'):
    nrows = int(np.ceil(len(list_of_columns)/2))  
    fig, ax = plt.subplots(nrows=nrows, ncols=2,figsize = (15,100)) 

    ax = ax.ravel()
    for i, column in enumerate(list_of_columns): 
        sns.boxplot(x=dataframe[column],y=dataframe[reference],ax=ax[i])

In [ ]:
subplot_boxplot(df,nominal_features)

## <span style="color: blue"> As seen from the countplot, SalePrice has increased steadily with Year Built</span>

In [ ]:
df['Year Built'].astype='int'

In [ ]:
plt.figure(figsize=(10,5))
ax = sns.countplot(df['Year Built'])

In [ ]:
def subplot_boxplot(dataframe, list_of_columns,reference='SalePrice'):
    nrows = int(np.ceil(len(list_of_columns)/2)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=2,figsize = (15,100)) 
    

    ax = ax.ravel()  
    for i, column in enumerate(list_of_columns):  
        sns.boxplot(x=dataframe[column],y=dataframe[reference],ax=ax[i])

In [ ]:
def subplot_countplot(dataframe, list_of_columns):
    nrows = int(np.ceil(len(list_of_columns)/2)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=2,figsize = (15,100))  

    ax = ax.ravel()  
    for i, column in enumerate(list_of_columns):  
        sns.countplot(x=dataframe[column],ax=ax[i])

In [ ]:
subplot_countplot(df,nominal_features)

### <span style="color: blue"> Since garage has many columns such as Garage yr built, Garage Finish, Garage cars, Garage Area, Garage Qual, Garage condition, there is a need to drop columns that has been used for transformation.</span>

In [ ]:
df.filter(regex='Year|Yr').describe()

In [ ]:
df.filter(regex='Year|Yr|Garage').isna().sum()

In [ ]:
df['Year Built'].dtypes

In [ ]:
df['Year Remod/Add'].dtypes

In [ ]:
df['garage_age'] = df.apply(lambda x: (x['Yr Sold'] - x['Year Remod/Add']) if type(x['Garage Yr Blt']) == 'nan' else (x['Yr Sold'] - x['Garage Yr Blt']) , axis=1)

In [ ]:
df.garage_age.sort_values(ascending=True).head()

In [ ]:
df.drop(columns=['Year Built','Year Remod/Add','Garage Yr Blt'])

## <span style="color: blue"> HEATMAP TO SHOW CORRELATION, PAIRPLOTS AND SCATTERPLOTS TO SHOW OUTLIERS AND REMOVE OUTLIERS.</span>

In [ ]:

corr = df[numerical_columns].corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(15, 15))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5},annot=True)

## FROM HEATMAP, GR LIV AREA IS IMPORTANT FEATURE. CERTAIN NUMERICAL CATEGORIES WITH LOW CORRELATION CAN BE DROPPED

In [ ]:
sns.pairplot(df[numerical_columns])


## <span style="color: blue"> There are 2 outliers that can be seen that is above 4000 sqft from Gr Liv Area</span>

In [ ]:
sns.scatterplot(x='Gr Liv Area',y='SalePrice',data=df)


In [ ]:
df = df[df['Gr Liv Area'] <= 4000]


In [ ]:

df[numerical_columns]=df[numerical_columns].replace(np.nan,0)

In [ ]:
df[nominal_features]=df[nominal_features].replace(np.nan,'missing')

In [ ]:
def subplot_scatterplot(dataframe, list_of_columns,reference='SalePrice'):
    nrows = int(np.ceil(len(list_of_columns)/2)) 
    fig, ax = plt.subplots(nrows=nrows, ncols=2,figsize = (15,100)) 

    ax = ax.ravel() 
    for i, column in enumerate(list_of_columns):
        sns.scatterplot(x=dataframe[column],y=dataframe[reference],ax=ax[i])

In [ ]:
subplot_scatterplot(df,numerical_columns)


In [ ]:
df[numerical_columns].isna().sum()

In [ ]:
combined_features = nominal_features + numerical_columns

In [ ]:

df[combined_features].to_csv('/Users/shaminimagandran/Desktop/moditrain.csv')

# <span style="color: red"> Preproceesing andFeature_Engineering</span>

In [ ]:
df1 =pd.read_csv('moditrain.csv')
df1

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df1.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
cat_features = ['Alley','Lot Shape','Land Contour',
                   'Overall Cond','Roof Style','Roof Matl','Exterior 1st',
                   'Exterior 2nd', 'Mas Vnr Type','Exter Qual',
                   'Exter Cond','Utilities','Lot Config','Land Slope',
                   'Neighborhood','Condition 1','Condition 2',
                   'Bldg Type','House Style','Overall Qual','Foundation','Bsmt Qual','Bsmt Cond',
                   'Bsmt Exposure','MS SubClass','MS Zoning','Street','BsmtFin Type 1','BsmtFin Type 2',
                   'Heating','Heating QC','Central Air','Electrical',
                   'Bsmt Full Bath','Bsmt Half Bath','Full Bath',
                   'Half Bath','Bedroom AbvGr','Kitchen AbvGr','Kitchen Qual',
                   'TotRms AbvGrd','Functional','Fireplaces',
                   'Fireplace Qu','Garage Type',
                   'Garage Finish','Garage Cars','Garage Qual',
                   'Garage Cond','Paved Drive','Pool QC','Fence',
                   'Misc Feature','Mo Sold',
                   'Sale Type','Year Built','Year Remod/Add','Garage Yr Blt','Yr Sold']

numerical_columns = ['Lot Frontage','Lot Area','Mas Vnr Area',
 'BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF',
 'Total Bsmt SF','1st Flr SF','2nd Flr SF',
 'Low Qual Fin SF','Gr Liv Area','Garage Area','Wood Deck SF',
 'Open Porch SF','Enclosed Porch','3Ssn Porch',
 'Screen Porch','Pool Area','Misc Val']



# <span style="color: red">COLUMNS FOR FEATURE SELECTION AND DROPPING OF COLUMNS</span>

In [ ]:
df1['outdoor_SF'] = df1['Wood Deck SF'] + df1['Open Porch SF'] + df1['Screen Porch']

In [ ]:
df1 = df1.drop(['Screen Porch','Wood Deck SF','Open Porch SF'],axis=1)

In [ ]:
df1.head(2)

In [ ]:
quality_codes = {
    'Ex' : 5,
    'Gd' : 4,
    'TA' : 3,
    'Fa' : 2,
    'Po' : 1,
    'NA' : 0,
    'missing' :0,
    np.nan : 0
}

In [ ]:
cat_quality_features = ['Exter Qual','Exter Cond','Bsmt Qual','Bsmt Cond','Heating QC','Kitchen Qual','Fireplace Qu',
                           'Garage Qual', 'Garage Cond','Pool QC']

In [ ]:
for feature in cat_quality_features:
    df1[feature] = df1[feature].map(quality_codes)

In [ ]:
df1['Bsmt Exposure'] = df1['Bsmt Exposure'].map({'Gd':5,'Av':4,'Mn':3,'No':2,'NA':1,'missing':0})

In [ ]:
df1['Garage Finish'] = df1['Garage Finish'].map({'Fin':3,'Rfn':2,'Unf':1,'NA':0,'missing':0})

In [ ]:
df1['Lot Shape'] = df1['Lot Shape'].map({'Reg':4,'IR1':3,'IR2':2,'IR3':1,'missing':0})


In [ ]:
df1['house_score'] = df1['Overall Qual'] + df1['Overall Cond']
df1['exter_score'] = df1['Exter Qual'] + df1['Exter Cond']
df1['fireplace_score'] = df1['Fireplace Qu'] * df1['Fireplaces']
df1['garage_score'] =  df1['Garage Qual'] + df1['Garage Cond'] + df1['Garage Finish'] 
df1['kitchen_score'] = df1['Kitchen AbvGr'] * df1['Kitchen Qual']
df1['total_baths'] = df1['Full Bath'] + df1['Half Bath']
df1['basement_score'] =   df1['Bsmt Cond'] + df1['Bsmt Qual']

In [ ]:
df1.head(2)

In [ ]:
aggregated_columns = ["house_score","exter_score","fireplace_score","garage_score","kitchen_score",
                     "total_baths","basement_score"]

In [ ]:
columns_to_drop = ["MS SubClass","MS Zoning","Street","Alley","Land Countour",
                  "Overall Qual","Overall Cond","Full Bath","Half Bath",
                   "Utilities","Land Slope","Condition 1",
                  "Condition 2","Bldg Type","Roof Matl",
                  "Roof Style","Exter Qual","Exter Cond",'Mas Vnr Type','Bsmt Cond',
                  "Bsmt Exposure","BsmtFin Type 1","Bsmt Qual","BsmtFin Type 2","Heating",
                  "Electrical","Bsmt Half Bath","Kitchen AbvGr","Kitchen Qual",
                  "Functional","Fireplace Qu","Fireplaces","Garage Qual","Garage Cond","Garage Finish","Garage Cars",
                  "Paved Drive","Pool QC","Fence","Misc Feature","Sale Type","Neighborhood","Foundation",
                  "Exterior 1st" , "Exterior 2nd","Bsmt Full Bath", "Garage Type" , "Lot Config"]

In [ ]:
feature_new = [col for col in cat_features if col not in columns_to_drop]


In [ ]:
feature_new.extend(aggregated_columns)

In [ ]:
feature_new

In [ ]:
feature_new.append('Gr Liv Area')

In [ ]:
feature_new.append('Mas Vnr Area')

In [ ]:
feature_new.append('Total Bsmt SF')

In [ ]:
feature_new.append('SalePrice') 


In [ ]:
corr_new = df1[feature_new].corr()


In [ ]:
plt.figure(figsize=(12, 9))

sns.heatmap(df1[feature_new].corr(), annot = True) 

In [ ]:
df1[feature_new].to_csv('featurenew.csv')


# <span style="color: green">   LINEAR, LASSO AND RIDGE REGRESSION MODELS </span>

## <span style="color: green"> TRAIN TEST SPLIT   </span>

In [ ]:
df2 = pd.read_csv('featurenew.csv')
df2.head(2)

In [ ]:
df2.columns

In [ ]:
df2.columns
df2.isnull().sum()

In [ ]:

y = df2['SalePrice']
features = [col for col in df2.columns if col != 'SalePrice']
X = df2[features] 

In [ ]:
X.columns

In [ ]:
X.isnull().sum().sort_values(ascending=False).head()

# <span style="color: green">  TRAIN TEST SPLIT AND CHECKING OF X- TRAIN AND X-TEST SHAPE</span>

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
X_train.columns

In [ ]:
new = df2.drop(columns=['Unnamed: 0','Central Air', 'Total Bsmt SF', 'Lot Shape', 'Land Contour','garage_score','House Style','Year Built', 'Year Remod/Add', 'Garage Yr Blt', 'Yr Sold','Mas Vnr Area', 'Lot Shape'], axis =1)
new.head()

In [ ]:
new.columns

In [ ]:
df2[new.columns].dtypes


In [ ]:
y = df2['SalePrice']
features = [col for col in df2[new.columns] if col != 'SalePrice']
X = df2[features]


In [ ]:
X.columns

## <span style="color: green">  TEST DATA PREPROCESSING  </span>

In [ ]:
test_model = pd.read_csv("test.csv")
test_model.head(2)

In [ ]:
test_model['outdoor_SF'] = test_model['Wood Deck SF'] + test_model['Open Porch SF'] + test_model['Screen Porch']

In [ ]:
test_model= test_model.drop(['Screen Porch','Wood Deck SF','Open Porch SF'],axis=1)

In [ ]:
quality_codes = {
    'Ex' : 5,
    'Gd' : 4,
    'TA' : 3,
    'Fa' : 2,
    'Po' : 1,
    'NA' : 0,
    'missing' :0,
    np.nan : 0
}

In [ ]:
test_cat_quality_features = ['Exter Qual','Exter Cond','Bsmt Qual','Bsmt Cond','Heating QC','Kitchen Qual','Fireplace Qu',
                           'Garage Qual', 'Garage Cond','Pool QC']

In [ ]:
for feature in test_cat_quality_features:
    test_model[feature] = test_model[feature].map(quality_codes)

In [ ]:
test_model['Bsmt Exposure'] = test_model['Bsmt Exposure'].map({'Gd':5,'Av':4,'Mn':3,'No':2,'NA':1,'missing':0})


In [ ]:
test_model['Garage Finish'] = test_model['Garage Finish'].map({'Fin':3,'Rfn':2,'Unf':1,'NA':0,'missing':0})


In [ ]:
test_model['Lot Shape'] = test_model['Lot Shape'].map({'Reg':4,'IR1':3,'IR2':2,'IR3':1,'missing':0})


In [ ]:
test_model['house_score'] = test_model['Overall Qual'] + test_model['Overall Cond']
test_model['exter_score'] = test_model['Exter Qual'] + test_model['Exter Cond']
test_model['fireplace_score'] = test_model['Fireplace Qu'] * test_model['Fireplaces']
test_model['garage_score'] =  test_model['Garage Qual'] + test_model['Garage Cond'] + test_model['Garage Finish'] 
test_model['kitchen_score'] = test_model['Kitchen AbvGr'] * test_model['Kitchen Qual']
test_model['total_baths'] = test_model['Full Bath'] + test_model['Half Bath']
test_model['basement_score'] =   test_model['Bsmt Cond'] + test_model['Bsmt Qual']

In [ ]:
test_aggregated_columns = ["house_score","exter_score","fireplace_score","garage_score","kitchen_score",
                     "total_baths","basement_score"]

In [ ]:
columns_to_drop = ["MS SubClass","MS Zoning","Street","Alley","Land Countour",
                  "Overall Qual","Overall Cond","Full Bath","Half Bath",
                   "Utilities","Land Slope","Condition 1",
                  "Condition 2","Bldg Type","Roof Matl",
                  "Roof Style","Exter Qual","Exter Cond",'Mas Vnr Type','Bsmt Cond',
                  "Bsmt Exposure","BsmtFin Type 1","Bsmt Qual","BsmtFin Type 2","Heating",
                  "Electrical","Bsmt Half Bath","Kitchen AbvGr","Kitchen Qual",
                  "Functional","Fireplace Qu","Fireplaces","Garage Qual","Garage Cond","Garage Finish","Garage Cars",
                  "Paved Drive","Pool QC","Fence","Misc Feature","Sale Type","Neighborhood","Foundation",
                  "Exterior 1st" , "Exterior 2nd","Bsmt Full Bath", "Garage Type" , "Lot Config"]

In [ ]:
test_feature_new = [col for col in test_cat_quality_features if col not in columns_to_drop]


In [ ]:
test_feature_new.extend(test_aggregated_columns)


In [ ]:
test_model[test_feature_new].to_csv('/Users/shaminimagandran/Desktop/testfeaturenew.csv')


## <span style="color: Magenta">   DEFINING X AND Y VARIABLES  </span>

In [ ]:
y = df2['SalePrice']
features = [col for col in df2[features] if col != 'SalePrice']
X = df2[features]


## <span style="color: Magenta">   The dataset was fairly messy with regard to the number of missing values and the way levels of categorical variables were organized and coded. In order to tackle the large amount of cleaning required, we thought it was best to split the data </span>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 43)

In [ ]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)


In [ ]:
ss = StandardScaler()
X_test_scaled = ss.fit_transform(X_test)


## <span style="color: MediumTurquoise">   LINEAR MODEL, FIT AND PREDICT </span>

In [ ]:
test_model = pd.read_csv("/Users/shaminimagandran/Desktop/test.csv")
test_model.head()


## <span style="color: Magenta"> Calculating R2 score </span>

###  This shows that despite the high number of predictors variables in the model  we have not been penalized heavily since most variables account for a significant amount of variance in sale price. We can further simplify and improve this model by dropping highly correlated variables and insignificant predictors.

In [ ]:
test_model.shape

In [ ]:
lr_submission = LinearRegression().fit(X_train_scaled,y_train)


In [ ]:
lr = LinearRegression()
model = lr.fit(X_train,y_train)
print(model.score(X_train,y_train))
cross_val_score(LinearRegression(), X_train, y_train,cv=4)


In [ ]:
print(model.score(X_test,y_test))
y_predict = model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_predict))


# <span style="color: Tomato">  Cross Validation and kfold </span>

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cross_val_score(lr, X_train_scaled, y_train, cv=kf).mean()


# <span style="color: RoyalBlue"> Ridge </span>

In [ ]:
ridge_model = Ridge()

In [ ]:
ridge_model = ridge_model.fit(X_train,y_train)

In [ ]:
y_predict_ridge = ridge_model.predict(X_test) 

In [ ]:
np.sqrt(mean_squared_error(y_test,y_predict_ridge))

 # <span style="color: DarkMagenta">  Lasso Model </span>

In [ ]:
lasso = Lasso()

In [ ]:
lasso_model = lasso.fit(X_train,y_train)


In [ ]:
y_predict_lasso = lasso_model.predict(X_test)


In [ ]:
np.sqrt(mean_squared_error(y_test,y_predict_lasso))


# <span style="color: blue"> Conclusion and Recommendations</span>


### RMSE results are better both on training and test sets.There are some Neighborhood categories that are interesting.House prices change a whole lot from one neighborhood to another in the same city.

### Putting time and effort into preparing the dataset and optimizing the  models resulted in a decent score. However, it is more important to have a more comprehensive features that includes more features on the house keeping in mind the more important features that home buyers are looking for. 

### The ridge model performed the best in giving the lowest error.

### My columns used in anlaysis: 'Heating QC', 'Bedroom AbvGr', 'TotRms AbvGrd', 'Mo Sold','house_score', 'exter_score', 'fireplace_score', 'kitchen_score','total_baths', 'basement_score', 'Gr Liv Area'

### This is a combination of some of the features of the house. The score can be improved however this is a more holistic view on all the aspects of the house.

### The biggest takeaways for me were that square footage, quality / condition, quality of materials, neighborhood, and number of bathrooms are very important factors for sale prices from correlation and heatmaps and value counts. However, other attributes should be considered for home owners looking to sell and home buyers looking to buy


# <span style="color: blue"> Limitations </span>

### I would recommend removing any houses with more than 4000 square feet from the data set (which eliminates these 5 unusual observations) before assigning it to students.
### There were many missing and nan values.If we have numeric data with a normal distribution, dealing with nulls can be relatively simple; we can impute mean or median values for the nulls. However, for this dataset, there are surprisingly few nulls in the numeric data. 
### There are also categorical data hence null values have to be addressed as well.
